## Tuần 5(3/10-9/10)

## Train model nhận diện sử dụng Mediapipe

### Tìm hiểu về cách nhận diện sử dụng Mediapipe

Một số video đã xem:

Video **American Sign Language Detection with Python and MediaPipe** của **Rob Mulla. Link video:** https://www.youtube.com/watch?v=L-IaQch8KYY

Video **Sign Language Detection Using Machine Learning | Python Project** của **Progress With Python. Link video:** https://www.youtube.com/watch?v=Ui85SVJsRf8

Video **Sign language detection with Python and Scikit Learn | Landmark detection | Computer vision tutorial** của **Computer vision engineer. Link video:** https://www.youtube.com/watch?v=MJCSjXepaAM

Video **Custom Hand Gesture Recognition with Hand Landmarks Using Google’s Mediapipe + OpenCV in Python của Ivan Goncharov. Link video:** https://www.youtube.com/watch?v=a99p_fAr6e4

### Thực hiện train model theo video **Hands gesture recognition with mediapipe and svm classifier của AI4LIFE**

**Link Video:** [Hands gesture recognition with mediapipe and svm classifier](https://www.youtube.com/watch?v=dlyVy_LNCEQ)

**Link Github:** [hand-gesture-recognition](https://github.com/dongdv95/hand-gesture-recognition)

Sau khi xem một vài video hướng dẫn nhận diện tay sử dụng Mediapipe thì em thử thực hiện model với video hướng dẫn này. Kết quả thu được model nhận diện tay có độ chính xác còn thấp.

File **get_image.py** dùng để thực hiện thu thập hình ảnh cho dataset với mỗi ký tự là 100 ảnh, 50 ảnh cho tay trái và 50 ảnh cho tay phải( trừ chữ A, B, C, D, E, F, G, H). Sau quá trình đó thì ta thu được một DATASET với 2600 ảnh.

File **get_data.py** dùng để thực hiện label tự động cho mỗi ảnh để thu được các landmark của từng ảnh và được tập hợp lại thành file **dataset.csv**.

File **train.ipynb** dùng để thực hiện train model là **model.pkl**.

Sau khi có model thực hiện chạy wecam detect với file **hands_gesture_recog.py**. Vì em cảm thấy thiếu Landmark Mediapipe khi chạy wecam để kiểm tra detect tay khi sử dụng Mediapipe thế nào nên em đã lên chatGPT để thêm vào code hiện thị Landmark khi chạy wecam.

Code:

```
        import cv2
        import mediapipe as mp
        import numpy as np
        import pickle

        # Initialize MediaPipe Hands module and drawing utilities
        mp_hands = mp.solutions.hands
        mp_drawing = mp.solutions.drawing_utils
        hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

        def image_processed(hand_img):
            """
            This function processes the input image and extracts hand landmarks
            using MediaPipe's Hand Landmark detection model.
            """
            # Convert BGR to RGB
            img_rgb = cv2.cvtColor(hand_img, cv2.COLOR_BGR2RGB)

            # Flip the image for mirror effect (optional)
            # img_flip = cv2.flip(img_rgb, 1)

            # Process the flipped RGB image with MediaPipe Hands
            output = hands.process(img_rgb)

            try:
                # Extract hand landmarks if present
                data = output.multi_hand_landmarks[0]
                data = str(data).strip().split('\n')

                # Filter out unnecessary data
                garbage = ['landmark {', '  visibility: 0.0', '  presence: 0.0', '}']
                without_garbage = [i.strip()[2:] for i in data if i not in garbage]

                # Convert to float for model compatibility
                clean = [float(i) for i in without_garbage]
                return clean, output.multi_hand_landmarks[0]  # Return both the clean data and landmarks for drawing
            except:
                # Return an array of zeros and None if no hand is detected
                return np.zeros([1, 63], dtype=int)[0], None

        # Load pre-trained SVM model
        with open('model.pkl', 'rb') as f:
            svm = pickle.load(f)

        # Initialize webcam
        cap = cv2.VideoCapture(0)

        if not cap.isOpened():
            print("Cannot open camera")
            exit()

        while True:
            # Capture frame-by-frame from webcam
            ret, frame = cap.read()

            if not ret:
                print("Can't receive frame (stream end?). Exiting ...")
                break

            # Process the frame to get hand landmarks and the clean data
            data, hand_landmarks = image_processed(frame)

            # Convert data to numpy array
            data = np.array(data)

            # Predict the hand sign using the pre-trained SVM model
            y_pred = svm.predict(data.reshape(-1, 63))
            print(y_pred)

            # Draw the MediaPipe landmarks on the frame if a hand is detected
            if hand_landmarks:
                # Draw landmarks and connections using MediaPipe's drawing utilities
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Font settings for the prediction text
            font = cv2.FONT_HERSHEY_SIMPLEX
            org = (50, 100)
            fontScale = 3
            color = (255, 0, 0)  # Blue color
            thickness = 5

            # Display the predicted hand sign on the frame
            frame = cv2.putText(frame, str(y_pred[0]), org, font, 
                                fontScale, color, thickness, cv2.LINE_AA)

            # Show the frame with landmarks and predictions
            cv2.imshow('Hand Sign Detection with MediaPipe', frame)

            # Break the loop on 'q' key press
            if cv2.waitKey(1) == ord('q'):
                break

        # Release the webcam and close windows
        cap.release()
        cv2.destroyAllWindows()

        # Close the MediaPipe hands instance
        hands.close()

```

**Link Video demo:** https://drive.google.com/file/d/1oXg2cvd-LwXO7xLRznvI59KoB1YpEK54/view?usp=sharing

### Một số thứ bên lề

Trong quá trình tìm hiểu Mediapipe thì em có tìm được một bài Github có nói về việc ghi lại ảnh động của ngôn ngữ ký hiệu sử dụng Mediapipe và DTW. Link Github: [Sign-Language-Recognition--MediaPipe-DTW](https://github.com/gabguerin/Sign-Language-Recognition--MediaPipe-DTW)